# TAREA. Dataset logs nasa

Arturo Sánchez Palacio


Fecha: 10/XII/18

El objetivo de esta práctica es emplear el RDD access_logs para realizar una serie de cálculos:

En primer lugar se deben cargar los datos en un contexto Spark:

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [3]:
data_file = "./apache.access.log_small"
raw_data = sc.textFile(data_file)

Hasta aquí se han cargado los datos en el RDD raw_data. Como primera medida para asegurar que los datos se han subido de manera correcta se procede al conteo de los mismos:

In [4]:
raw_data.count()

3432

Además vamos a visualizar los primero elementos del RDD:

In [5]:
raw_data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

Una vez confirmado que la carga ha sido realizada de manera correcta se procede al parseado de los datos:

En el parseado se emplea la expresión regular vista hoy en clase:

In [6]:
import re
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1
n_logs = raw_data.count()

In [7]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [8]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0])
parsed_def = parsed_rdd.map(lambda line: map_log(line))

A continuación se presentan las estadísticas (incluyendo mínimo, máximo y media) del tamaño de las peticiones:

In [9]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats()

(count: 3432, mean: 16051.863636363634, stdev: 53247.8157482, max: 887988.0, min: 0.0)

El siguiente objetivo es calcular el número de peticiones de cada código de respuesta:

In [10]:
n_codes = parsed_def.map(lambda line: (line[-2], 1)).distinct().count()
codes_count = (parsed_def.map(lambda line: (line[-2], 1))
          .reduceByKey(lambda a, b: a + b)
          .takeOrdered(n_codes, lambda x: -x[1]))
codes_count

[('200', 3140), ('304', 219), ('302', 50), ('404', 22), ('403', 1)]

__Solución:__ Así el código 200 aparece para 3140 logs, el 304 para 219 logs, el 302 para 50 logs, el 404 para 22 logs y el 403 para un único log.

Si se desean ver  20 hosts visitados más de diez veces:

In [11]:
result = parsed_def.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b).takeOrdered(20, lambda x: -x[1])
result

[('ix-min1-02.ix.netcom.com', 78),
 ('uplherc.upl.com', 71),
 ('port26.ts1.msstate.edu', 59),
 ('h96-158.ccnet.com', 56),
 ('in24.inetnebr.com', 55),
 ('thing1.cchem.berkeley.edu', 54),
 ('piweba3y.prodigy.com', 54),
 ('adam.tower.com.au', 44),
 ('ip55.van2.pacifier.com', 43),
 ('ppp1016.po.iijnet.or.jp', 41),
 ('hsccs_gatorbox07.unm.edu', 40),
 ('www-b2.proxy.aol.com', 40),
 ('www-d1.proxy.aol.com', 39),
 ('port13.wavenet.com', 37),
 ('133.43.96.45', 37),
 ('pc-heh.icl.dk', 33),
 ('haraway.ucet.ufl.edu', 32),
 ('193.84.66.147', 31),
 ('www-c1.proxy.aol.com', 30),
 ('143.158.26.50', 29)]

A continuación se muestran los 10 endpoints más visitados:

In [12]:
result = parsed_def.map(lambda line: (line[6],1)).reduceByKey(lambda a, b: a + b).takeOrdered(10, lambda x: -x[1])
result

[('/images/KSC-logosmall.gif', 167),
 ('/images/NASA-logosmall.gif', 160),
 ('/images/MOSAIC-logosmall.gif', 122),
 ('/images/WORLD-logosmall.gif', 120),
 ('/images/USA-logosmall.gif', 118),
 ('/images/ksclogo-medium.gif', 106),
 ('/', 85),
 ('/history/apollo/images/apollo-logo1.gif', 74),
 ('/images/launch-logo.gif', 69),
 ('/images/ksclogosmall.gif', 66)]

Si se desean los 10 endpoints más visitados que no han devuelto un resultado (es decir, que tienen un código distinto de 200):

In [13]:
result = (parsed_def.filter(lambda line: line[9] != '200')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b)
          .takeOrdered(10, lambda x: -x[1]))
result

[('/images/NASA-logosmall.gif', 25),
 ('/images/KSC-logosmall.gif', 24),
 ('/images/MOSAIC-logosmall.gif', 17),
 ('/images/WORLD-logosmall.gif', 17),
 ('/images/USA-logosmall.gif', 16),
 ('/images/ksclogo-medium.gif', 10),
 ('/images/construct.gif', 8),
 ('/software/winvn/winvn.html', 8),
 ('/software/winvn/bluemarb.gif', 8),
 ('/software/winvn/wvsmall.gif', 6)]

A continuación se calcula el número de hosts distintos:

In [14]:
parsed_def.map(lambda line: line[0]).distinct().count()

311

Tras ello se puede calcular el número de hosts únicos cada día:

In [15]:
from datetime import datetime
def day_month(line):
    date_time = line[3]
    return datetime.strptime(date_time[:11], "%d/%b/%Y") #Se parsea la fecha para trabajar con ella tal y como vimos en clase.
result = parsed_def.map(lambda line:  (day_month(line), 1)).reduceByKey(lambda a, b: a + b).distinct().collect()
result



[(datetime.datetime(1995, 8, 1, 0, 0), 3432)]

__Nota:__ En este dataset solo se poseén datos de un día por esa razón se obtiene este resultado.

Tras esto se pide calcular la media de peticiones diaria por host.

In [16]:
import pandas as pd

In [17]:
unique_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1]))))

length_result = (parsed_def.map(lambda line:  (day_month(line),line[0]))
          .groupByKey().mapValues(len))

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df = pd.DataFrame(day_count_dct )


El panda en este caso no se usa para realizar ningún cálculo si no simplemente para una mejor visualización.

__Solución:__ 

In [18]:
day_count_df

,Día,Media
0,1995-08-01,11.03537


A continuación, se muestra una lista de 40 endpoints que generan código de respuesta = 404:

In [19]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(40, lambda x: -x[1]))
result

[('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/pub/winvn/readme.txt', 2),
 ('/www/software/winvn/winvn.html', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/history/history.htm', 1),
 ('/history/apollo/a-004/movies/', 1)]

__Nota:__ En la lista solo aparecen 13 endpoints porque se está trabajando con la base de datos reducida.

El siguiente comando mostraría el top 25 de endpoints que más códigos 404 generan de dispnerse de más de 13:

In [20]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(25, lambda x: -x[1]))
result

[('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/pub/winvn/readme.txt', 2),
 ('/www/software/winvn/winvn.html', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/history/history.htm', 1),
 ('/history/apollo/a-004/movies/', 1)]

Si se desean obtener el top 5 de días que más códigos de error 404 generan:

In [21]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line:  (day_month(line), 1))
          .reduceByKey(lambda a, b: a+b).collect())
day = [x[0] for x in result]
count = [x[1] for x in result]
day_count_dct = {'day':day, 'count':count}
day_count_df = pd.DataFrame(day_count_dct )


De nuevo el paso a DF se emplea únicamente para una mejor visualización:

In [22]:
day_count_df.sort_values('count', ascending = False)[:10]


,count,day
0,22,1995-08-01


__Nota:__ De nuevo se obtiene un solo día por trabajarse con la base de datos reducida.